In [1]:
from tqdm import tqdm
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Код товара], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Товарное направление],
[Товарная группа], [Товарный кластер], [Проба], [Размер изделия], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Дизайн], [Бренд], [Гендерный признак], [Ценовая корзина], [Группа цен], [Остаток], [Цена Розн., за шт],
[Чистый вес], [Цена изделия на бирке], [Скидка на изделии], [Запрет скидки], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)], [Дата закупки]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'СИ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')

In [4]:
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')

In [5]:
kniga = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Книга11.xlsx')

In [6]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['2024-03-10_2024-03-19.xlsx',
 '2024-03-20_2024-03-31.xlsx',
 '2024-04-01_2024-04-09.xlsx',
 '2024-04-10_2024-04-19.xlsx',
 '2024-04-20_2024-04-30.xlsx',
 '2024-05-01_2024-05-09.xlsx',
 '2024-05-10_2024-05-19.xlsx',
 '2024-05-20_2024-05-31.xlsx',
 '2024-06-01_2024-06-09.xlsx',
 '2024-06-10_2024-06-10.xlsx',
 '2024-06-10_2024-06-12.xlsx',
 '2024-06-10_2024-06-18.xlsx']

In [7]:
sale = pd.DataFrame()
for name in tqdm(all_filenames):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

100%|██████████| 12/12 [11:51<00:00, 59.33s/it]


In [8]:
sale['Дата'] = pd.to_datetime(sale['Дата'], dayfirst=True, format='mixed')

In [9]:
sale = sale[(sale['Дата'] >= pd.to_datetime('2024-03-14', dayfirst=True, format='mixed')) & 
            (sale['Дата'] <= pd.to_datetime('2024-06-19', dayfirst=True, format='mixed'))]

In [10]:
kniga['Код склада'] = kniga['Код склада'].astype(str)
skladi = kniga['Код склада'].unique().tolist()

In [11]:
sale.columns

Index(['КБК', 'Дата', 'Продавец', 'Номер чека', 'ID продажи', 'Тов',
       'Артикул поставщика', 'Количество', 'Номенклатура', 'Номер карты',
       'Товарная группа', 'Товарное направление', 'Тип 1',
       'Списание СберСпасибо', 'Группа наценки', 'Вес', 'Проба', 'Размер',
       'ШК', 'Цена без скидки', 'Базовая скидка', 'Скидка в руб.', 'Название',
       'Цена после скидки', 'Phone', 'Себестоимость', 'Сумма', 'Услуга',
       'Общая сумма'],
      dtype='object')

In [12]:
ids = stock[['Код товара', 'ID Сайта']]
ids.columns = ['Тов', 'ID Сайта']

In [13]:
sale = sale.merge(ids, how='left', on='Тов')
sale = sale.drop(columns=['Тов'])
sale = sale[['ID Сайта', 'Количество', 'Общая сумма']]

MemoryError: Unable to allocate 8.22 GiB for an array with shape (3, 367541283) and data type object

In [ ]:
выаывав
sale = sale.groupby(by = 'ID Сайта', as_index=False)[['Количество', 'Общая сумма']].sum()

In [14]:
stock.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток',
       'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Дата закупки'],
      dtype='object')

In [15]:
stock.columns = ['code', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток', 'Цена Розн., за шт',
    'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии',
    'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
    'Дата закупки']

In [16]:
stock = stock.query('code in @skladi')

In [17]:
stock.columns = ['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток', 'Цена Розн., за шт',
    'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии',
    'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
    'Дата закупки']

In [18]:
result = stock.drop_duplicates(subset=['ID Сайта'])

In [19]:
tovs = result['ID Сайта'].dropna().tolist()

In [20]:
images['article'] = images['article'].astype(str)

In [21]:
images = images.query('article in @tovs')

In [22]:
images = images[['article', 'published', 'Ссылка на изображение']]

In [23]:
images.columns = ['ID Сайта', 'Опубликован', 'Ссылка на изображение']

In [24]:
result = result.merge(images, how='outer', on='ID Сайта')

In [25]:
result = result.drop(columns = ['Остаток'])

In [26]:
df = stock

In [27]:
stock = df.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()

In [28]:
result = result.merge(stock, how='left', on='ID Сайта')

In [29]:
sale.columns = ['ID Сайта', 'Количество продаж 14.03-19.06', 'Сумма продаж 14.03-19.06']

In [30]:
result = result.merge(sale, how='left', on='ID Сайта')

In [31]:
result.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Дата закупки',
       'Опубликован', 'Ссылка на изображение', 'Остаток',
       'Количество продаж 14.03-19.06', 'Сумма продаж 14.03-19.06'],
      dtype='object')

In [32]:
result = result.drop_duplicates(subset=['ID Сайта'])

In [33]:
df['Чистый вес'] = df['Чистый вес'].astype(float)

In [36]:
clear_weight = df.groupby(by = 'ID Сайта', as_index=False)['Чистый вес'].agg({'min', 'median', 'max'})
clear_weight.columns = ['ID Сайта', 'Чистый вес макс', 'Чистый вес мин', 'Чистый вес медиана']
clear_weight['Чистый вес размах, %'] = (clear_weight['Чистый вес макс'] / clear_weight['Чистый вес мин'] - 1) 

In [37]:
clear_weight

,ID Сайта,Чистый вес макс,Чистый вес мин,Чистый вес медиана,"Чистый вес размах, %"
0,,94.789,0.773,5.7695,121.624838
1,1000030,2.280,2.149,2.2100,0.060959
2,1000181,4.230,2.981,3.8455,0.418987
3,1000188,1.480,1.145,1.2640,0.292576
4,1000336,6.951,5.751,6.3515,0.208659
...,...,...,...,...,...
46162,8380059,2.930,2.810,2.8400,0.042705
46163,8380065,2.880,2.870,2.8700,0.003484
46164,8380085,2.390,2.360,2.3750,0.012712
46165,8380087,2.370,2.360,2.3650,0.004237


In [38]:
result = result.merge(clear_weight, how='left', on='ID Сайта')

In [39]:
result = result[result['ID Сайта'] != '']
df = df[df['ID Сайта'] != '']

In [42]:
price_birka = df.groupby(by = 'ID Сайта', as_index=False)['Цена Розн., за шт'].agg({'min', 'median', 'max'})
price_birka.columns = ['ID Сайта', 'Цена изделия на бирке макс', 'Цена изделия на бирке мин', 'Цена изделия на бирке медиана']

In [45]:
price_birka.sample(5)

,ID Сайта,Цена изделия на бирке макс,Цена изделия на бирке мин,Цена изделия на бирке медиана
17840,2218015,2026.0,2026.0,2026.0
7404,1459017,3315.0,3314.0,3315.0
40184,7237881,3826.0,3826.0,3826.0
14531,1906962,2654.0,2654.0,2654.0
15925,1990511,7437.0,4290.0,4648.0


In [46]:
result = result.merge(price_birka, how='left', on='ID Сайта')

In [47]:
base_discount = df.groupby(by = 'ID Сайта', as_index=False)['Скидка на изделии'].agg({'min', 'max'})
base_discount.columns = ['ID Сайта', 'Базовая скидка макс', 'Базовая скидка мин']

In [48]:
base_discount

,ID Сайта,Базовая скидка макс,Базовая скидка мин
0,1000030,40.0,40.0
1,1000181,40.0,40.0
2,1000188,40.0,40.0
3,1000336,40.0,40.0
4,1000521,40.0,40.0
...,...,...,...
46161,8380059,40.0,0.0
46162,8380065,40.0,30.0
46163,8380085,40.0,30.0
46164,8380087,40.0,30.0


In [49]:
result = result.merge(base_discount, how='left', on='ID Сайта')

In [50]:
df['Цена со скидками'] = np.where(
    df['Запрет скидки'] == 'Да', df['Цена Розн., за шт'], df['Цена Розн., за шт'] * (1-df['Скидка на изделии']/100) * 0.8 * 0.6
)

In [53]:
price_skidki = df.groupby(by = 'ID Сайта', as_index=False)['Цена со скидками'].agg({'min', 'median', 'max'})
price_skidki.columns = ['ID Сайта', 'Цена со скидками макс', 'Цена со скидками мин', 'Цена со скидками медиана']
price_skidki['Цена со скидками размах, %'] = (price_skidki['Цена со скидками макс'] / price_skidki['Цена со скидками мин'] - 1) 

In [54]:
price_skidki

,ID Сайта,Цена со скидками макс,Цена со скидками мин,Цена со скидками медиана,"Цена со скидками размах, %"
0,1000030,1308.096,1308.096,1308.096,0.000000
1,1000181,570.816,570.816,570.816,0.000000
2,1000188,570.528,570.528,570.528,0.000000
3,1000336,2395.296,2395.008,2395.296,0.000120
4,1000521,1195.200,1195.200,1195.200,0.000000
...,...,...,...,...,...
46161,8380059,3020.640,345.600,345.600,7.740278
46162,8380065,396.480,339.840,339.840,0.166667
46163,8380085,235.200,201.600,218.400,0.166667
46164,8380087,235.200,201.600,218.400,0.166667


In [55]:
result = result.merge(price_skidki, how='left', on='ID Сайта')

In [58]:
selfprice = df.groupby(by = 'ID Сайта', as_index=False)['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'].agg({'min', 'median', 'max'})
selfprice.columns = ['ID Сайта', 'Себестоимость макс', 'Себестоимость мин', 'Себестоимость медиана']
selfprice['Себестоимость размах, %'] = (selfprice['Себестоимость макс'] / selfprice['Себестоимость мин'] - 1) 

In [59]:
selfprice

,ID Сайта,Себестоимость макс,Себестоимость мин,Себестоимость медиана,"Себестоимость размах, %"
0,1000030,1064.66,1011.64,1036.100,0.052410
1,1000181,380.70,268.20,346.050,0.419463
2,1000188,410.05,219.58,333.775,0.867429
3,1000336,1807.00,1495.00,1651.000,0.208696
4,1000521,1393.00,1393.00,1393.000,0.000000
...,...,...,...,...,...
46161,8380059,553.51,542.17,547.000,0.020916
46162,8380065,529.00,529.00,529.000,0.000000
46163,8380085,321.77,317.73,319.750,0.012715
46164,8380087,321.54,320.00,320.770,0.004813


In [60]:
result = result.merge(selfprice, how='left', on='ID Сайта')

In [61]:
df['Наценка'] = df['Цена со скидками'] / df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] - 1

In [64]:
markup = df.groupby(by = 'ID Сайта', as_index=False)['Наценка'].agg({'min', 'median', 'max'})
markup.columns = ['ID Сайта', 'Наценка макс', 'Наценка мин', 'Наценка медиана']

In [65]:
markup

,ID Сайта,Наценка макс,Наценка мин,Наценка медиана
0,1000030,0.293045,0.228651,0.262519
1,1000181,1.128322,0.499385,0.649522
2,1000188,1.598269,0.391362,0.709320
3,1000336,0.602205,0.325565,0.450830
4,1000521,-0.141996,-0.141996,-0.141996
...,...,...,...,...
46161,8380059,4.504082,-0.370262,-0.366388
46162,8380065,-0.250510,-0.357580,-0.357580
46163,8380085,-0.259749,-0.373466,-0.316607
46164,8380087,-0.268520,-0.370000,-0.319260


In [66]:
result = result.merge(markup, how='left', on='ID Сайта')

In [67]:
zapret['Артикул сайта'] = zapret['Артикул сайта'].astype(str)

In [68]:
zapret = zapret['Артикул сайта'].tolist()

In [69]:
result['Запрет к размещению'] = np.nan
for i in result.index:
    if result['ID Сайта'][i] in zapret:
        result['Запрет к размещению'][i] = True
    else:
        result['Запрет к размещению'][i] = False

In [70]:
ecom = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\Остатки 2024.06.19.xlsx')

In [71]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

discount_ecom = pd.read_excel(latest_file)

In [72]:
discount_ecom.columns = ['article', 'Опубликован', 'discount_value_promotions_1', 'discount_value_promotions_2', 'discount_value_promotions_3', 'discount_value_promotions_4']

In [73]:
discount_ecom = discount_ecom.drop(columns=['discount_value_promotions_2', 'discount_value_promotions_3', 'discount_value_promotions_4'])

In [74]:
ecom = ecom.merge(discount_ecom, how='left', on='article')

In [75]:
def discount_index(discount):
    try:
        return 1 - discount / 100
    except:
        return 1

In [76]:
ecom.discount_value_promotions_1 = ecom.discount_value_promotions_1.apply(discount_index)

In [77]:
ecom['Цена со скидками Ecom'] = np.where(
    (ecom['price'] == ecom['discount_price']) & (ecom['discount_value_promotions_1'] == 1), ecom['discount_price'] * ecom['discount_value_promotions_1'], 
    np.where(
        ecom['discount_value_promotions_1'] == 0.5, ecom['discount_price'] * ecom['discount_value_promotions_1'], ecom['discount_price'] * ecom['discount_value_promotions_1'] * 0.8
    )
)

In [78]:
ecom['article'] = ecom['article'].astype(str)
ecom = ecom[['article', 'size', 'Цена со скидками Ecom']]

In [79]:
ecom = ecom.query('article in @tovs')

In [80]:
ecom.columns = ['ID Сайта', 'size', 'Цена со скидками Ecom']

In [81]:
result = result[result['ID Сайта'] != '']

In [82]:
result['ID Сайта'] = result['ID Сайта'].astype(int).astype(str)
result['Размер изделия'] = result['Размер изделия'].astype(str)

In [83]:
ecom['size'] = ecom['size'].fillna(0.0)
ecom['size'] = ecom['size'].astype(str)

In [84]:
ecom['merge'] = ecom['ID Сайта'] + ecom['size']

In [85]:
result['merge'] = result['ID Сайта'] + result['Размер изделия']

In [86]:
ecom = ecom.drop(columns=['ID Сайта', 'size'])

In [87]:
result = result.merge(ecom, how='outer', on='merge')

In [88]:
result = result.drop(columns=['merge'])

In [89]:
result = result.dropna(subset=['ID Сайта'])

In [90]:
df['Сумма в себестоимости на остатках'] = df['Остаток'] * df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [91]:
summa_ost = df.groupby(by ='ID Сайта', as_index=False)['Сумма в себестоимости на остатках'].sum()

In [92]:
result = result.merge(summa_ost, how='left', on='ID Сайта')

In [93]:
df['Цена за грамм'] = df['Цена со скидками'] / df['Чистый вес']

In [94]:
price_per_gramm = df.groupby(by = 'ID Сайта', as_index=False)['Цена за грамм'].agg({'min', 'median', 'max'})
price_per_gramm.columns = ['ID Сайта', 'Цена за грамм макс', 'Цена за грамм мин', 'Цена за грамм медиана']

In [95]:
price_per_gramm

,ID Сайта,Цена за грамм макс,Цена за грамм мин,Цена за грамм медиана
0,1000030,608.699860,573.726316,591.898643
1,1000181,191.484737,134.944681,148.437698
2,1000188,498.277729,385.491892,451.367089
3,1000336,416.500782,344.597324,377.125429
4,1000521,323.027027,323.027027,323.027027
...,...,...,...,...
46161,8380059,1059.873684,119.172414,122.120141
46162,8380065,138.146341,118.000000,118.411150
46163,8380085,99.661017,84.351464,92.006241
46164,8380087,99.240506,85.423729,92.332118


In [96]:
result['Себестоимость размах, %'].value_counts()

Себестоимость размах, %
0.000000    12363
0.062500       42
0.111111       31
0.333333       31
0.058824       28
            ...  
0.349747        1
0.520000        1
0.167173        1
0.624491        1
0.018265        1
Name: count, Length: 29639, dtype: int64

In [97]:
result = result.merge(price_per_gramm, how='left', on='ID Сайта')

In [98]:
result.to_csv('result.csv', index=False)

In [99]:
zapret_sk = df.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

In [100]:
yes = df[df['Запрет скидки'] == 'Да']
no = df[df['Запрет скидки'] == 'Нет']

In [101]:
yes = yes.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()
no = no.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

In [102]:
yes.columns = ['ID Сайта', 'Количество с запрещенной скидкой']

In [103]:
no.columns = ['ID Сайта', 'Количество с разрешенной скидкой']

In [104]:
yes = yes.merge(no, how='outer', on='ID Сайта')

In [105]:
result = result.merge(yes, how='left', on='ID Сайта')

In [106]:
result = result.drop_duplicates(subset=['ID Сайта'])

In [107]:
df['Дата закупки'] = pd.to_datetime(df['Дата закупки'], dayfirst=True, format='mixed')

In [108]:
df['Старая закупка (до 31.12.2022)'] = np.where(
    df['Дата закупки'] <= pd.to_datetime('2022-12-31', dayfirst=True, format='mixed'), 1, 0
)

In [109]:
df['Новая закупка (с 01.01.2023)'] = np.where(
    df['Дата закупки'] >= pd.to_datetime('2023-01-01', dayfirst=True, format='mixed'), 1, 0
)

In [110]:
data = df.groupby(by = 'ID Сайта', as_index=False)[['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)']].sum()

In [111]:
data['Дата закупки'] = np.where(
    (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые и новые',
    np.where(
        (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] == 0), 'Новые', 
        np.where(
            (data['Новая закупка (с 01.01.2023)'] == 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые', 'Старые'
        )
    )
)

In [112]:
data = data.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)'])

In [113]:
result = result.merge(data, how='left', on='ID Сайта')

In [114]:
result = result.drop(columns=['Дата закупки_x'])

In [115]:
result.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Опубликован',
       'Ссылка на изображение', 'Остаток', 'Количество продаж 14.03-19.06',
       'Сумма продаж 14.03-19.06', 'Чистый вес макс', 'Чистый вес мин',
       'Чистый вес медиана', 'Чистый вес размах, %',
       'Цена изделия на бирке макс', 'Цена изделия на бирке мин',
       'Цена изделия на бирке медиана', 'Базовая скидка макс',
       'Базовая скидка мин', 'Цена со скидками макс', 'Цена со скидками мин',
       'Цена со скидками медиана', 'Цена со скидками размах, %',
       'С

In [116]:
dsfdsfsd

NameError: name 'dsfdsfsd' is not defined

In [117]:
result.columns = ['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен',
    'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
    'Скидка на изделии', 'Запрет скидки',
    'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Опубликован',
    'Ссылка на изображение', 'Остаток', 'Количество продаж 14.03-19.06',
    'Сумма продаж 14.03-19.06', 'Чистый вес макс', 'Чистый вес мин',
    'Чистый вес медиана', 'Чистый вес размах, %',
    'Цена изделия на бирке макс', 'Цена изделия на бирке мин',
    'Цена изделия на бирке медиана', 'Базовая скидка макс',
    'Базовая скидка мин', 'Цена со скидками макс', 'Цена со скидками мин',
    'Цена со скидками медиана', 'Цена со скидками размах, %',
    'Себестоимость макс', 'Себестоимость мин', 'Себестоимость медиана',
    'Себестоимость размах, %', 'Наценка макс', 'Наценка мин',
    'Наценка медиана', 'Запрет к размещению', 'Цена со скидками Ecom',
    'Сумма в себестоимости на остатках', 'Цена за грамм макс',
    'Цена за грамм мин', 'Цена за грамм медиана',
    'Количество с запрещенной скидкой', 'Количество с разрешенной скидкой',
    'Дата закупки']

In [118]:
result[['Чистый вес размах, %']]

,"Чистый вес размах, %"
0,0.693767
1,0.671159
2,0.729958
3,0.617122
4,0.971477
...,...
46161,0.000000
46162,0.000000
46163,0.000000
46164,0.000000


In [120]:
result['Размах по весу'] = np.where(
    result['Чистый вес размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Чистый вес размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

In [121]:
result['Размах по себестоимости'] = np.where(
    result['Себестоимость размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Себестоимость размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

In [122]:
result['Размах по цене со скидками'] = np.where(
    result['Цена со скидками размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена со скидками размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

In [123]:
result['Цена за грамм размах, %'] = (result['Цена за грамм макс'] / result['Цена за грамм мин'] - 1) 

In [124]:
result['Размах по цене за грамм'] = np.where(
    result['Цена за грамм размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Цена за грамм размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

In [125]:
result['Скидка разрешена'] = result['Количество с запрещенной скидкой'] + result['Количество с разрешенной скидкой']

In [126]:
result['Скидка разрешена'] = np.where(
    result['Скидка разрешена'].astype(str).str.contains('nan'), 'Да или нет', 'Да и нет'
)

In [127]:
result.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Опубликован',
       'Ссылка на изображение', 'Остаток', 'Количество продаж 14.03-19.06',
       'Сумма продаж 14.03-19.06', 'Чистый вес макс', 'Чистый вес мин',
       'Чистый вес медиана', 'Чистый вес размах, %',
       'Цена изделия на бирке макс', 'Цена изделия на бирке мин',
       'Цена изделия на бирке медиана', 'Базовая скидка макс',
       'Базовая скидка мин', 'Цена со скидками макс', 'Цена со скидками мин',
       'Цена со скидками медиана', 'Цена со скидками размах, %',
       'С

In [128]:
result['Цена изделия на бирке размах, %'] = (result['Цена изделия на бирке макс'] / result['Цена изделия на бирке мин'] - 1) 

In [129]:
result['Размах цена изделия на бирке'] = np.where(
    result['Цена изделия на бирке размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена изделия на бирке размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<20%)'
    )
)

In [130]:
result = result.drop(columns = ['Код склада', 'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии', 'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
                                'Запрет скидки'])

In [131]:
tov = result.groupby(by = 'Товарная группа')['Остаток'].sum()

In [132]:
tov

Товарная группа
СИ АКСЕССУАР            30307.0
СИ БРАСЛЕТ БК          153323.0
СИ БРАСЛЕТ БК Декор     35553.0
СИ БРАСЛЕТ СК           37800.0
СИ БРАСЛЕТ ЦВ/К         16998.0
СИ ЖЕМЧУГ               56238.0
СИ КЕРАМИКА             27257.0
СИ КОЛЬЕ БК              6680.0
СИ КОЛЬЕ С ЭМАЛЬЮ        1215.0
СИ КОЛЬЕ СК             46415.0
СИ КОЛЬЕ ЦВ/К            7207.0
СИ КОЛЬЦО БК           116861.0
СИ КОЛЬЦО ИФ           330453.0
СИ КОЛЬЦО ПЕЧАТ         47963.0
СИ КОЛЬЦО ЦВ/К         105519.0
СИ ОБРУЧАЛЬНОЕ          51680.0
СИ ПОДАРКИ КРОСС        28737.0
СИ ПОДВЕС БК           134121.0
СИ ПОДВЕС ИФ           121652.0
СИ ПОДВЕС ЦВ/К          39882.0
СИ ПОСУДА               10182.0
СИ СЕРЬГИ БК            40570.0
СИ СЕРЬГИ ИФ           141813.0
СИ СЕРЬГИ РАЗН         286959.0
СИ СЕРЬГИ  ЦВ/К         94329.0
СИ ЦЕПЬ                412637.0
СИ ШАРМЫ                60132.0
СИ ЭМАЛЬ                17063.0
СИ ЯНТАРЬ               33709.0
Name: Остаток, dtype: float64

In [140]:
tov_sales = result.groupby(by = 'Товарная группа')['Количество продаж 14.03-19.06'].sum()

In [141]:
tov_sales

Товарная группа
СИ АКСЕССУАР            6244.0
СИ БРАСЛЕТ БК           9255.0
СИ БРАСЛЕТ БК Декор     8996.0
СИ БРАСЛЕТ СК           7845.0
СИ БРАСЛЕТ ЦВ/К         2034.0
СИ ЖЕМЧУГ               6481.0
СИ КЕРАМИКА             1344.0
СИ КОЛЬЕ БК              399.0
СИ КОЛЬЕ С ЭМАЛЬЮ        100.0
СИ КОЛЬЕ СК             4438.0
СИ КОЛЬЕ ЦВ/К            579.0
СИ КОЛЬЦО БК            2584.0
СИ КОЛЬЦО ИФ           10463.0
СИ КОЛЬЦО ПЕЧАТ         2669.0
СИ КОЛЬЦО ЦВ/К          4179.0
СИ ОБРУЧАЛЬНОЕ           300.0
СИ ПОДАРКИ КРОСС       13713.0
СИ ПОДВЕС БК           41096.0
СИ ПОДВЕС ИФ           30348.0
СИ ПОДВЕС ЦВ/К          6773.0
СИ ПОСУДА               3167.0
СИ СЕРЬГИ БК            8193.0
СИ СЕРЬГИ ИФ           33175.0
СИ СЕРЬГИ РАЗН         70633.0
СИ СЕРЬГИ  ЦВ/К        15028.0
СИ ЦЕПЬ                29770.0
СИ ШАРМЫ                6283.0
СИ ЭМАЛЬ                1338.0
СИ ЯНТАРЬ               3741.0
Name: Количество продаж 14.03-19.06, dtype: float64

In [134]:
result['Доля % остатка ID отосительно остатка ТГ'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % остатка ID отосительно остатка ТГ'] = result['Остаток'] / tov[t] 

In [135]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46166 entries, 0 to 46165
Data columns (total 61 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Код товара                                46166 non-null  object 
 1   Артикул поставщика                        46166 non-null  object 
 2   Внутренний артикул                        46166 non-null  object 
 3   ID Сайта                                  46166 non-null  object 
 4   Товарное направление                      46166 non-null  object 
 5   Товарная группа                           46166 non-null  object 
 6   Товарный кластер                          46166 non-null  object 
 7   Проба                                     46166 non-null  object 
 8   Размер изделия                            46166 non-null  object 
 9   Тип изделия 1                             46166 non-null  object 
 10  Тип изделия 2                     

In [136]:
result['Доля % относительно продаж ТГ 14.03-19.06'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % относительно продаж ТГ 14.03-19.06'] = result['Количество продаж 14.03-19.06'] / tov_sales[t] 

In [137]:
result[['Цена за грамм макс', 'Цена за грамм медиана', 'Цена за грамм мин',
    'Цена за грамм размах, %']]

,Цена за грамм макс,Цена за грамм медиана,Цена за грамм мин,"Цена за грамм размах, %"
0,616.827586,472.071156,297.216000,1.075351
1,612.136882,472.071156,299.612903,1.043093
2,696.289157,470.874525,302.048780,1.305221
3,564.356718,416.702371,348.988235,0.617122
4,720.724832,441.018480,345.039416,1.088819
...,...,...,...,...
46161,154.192208,154.192208,154.192208,0.000000
46162,190.843373,190.843373,190.843373,0.000000
46163,247.129909,247.129909,247.129909,0.000000
46164,433.333333,433.333333,433.333333,0.000000


In [139]:
result

,Код товара,Артикул поставщика,Внутренний артикул,ID Сайта,Товарное направление,Товарная группа,Товарный кластер,Проба,Размер изделия,Тип изделия 1,...,Размах по весу,Размах по себестоимости,Размах по цене со скидками,"Цена за грамм размах, %",Размах по цене за грамм,Скидка разрешена,"Цена изделия на бирке размах, %",Размах цена изделия на бирке,Доля % остатка ID отосительно остатка ТГ,Доля % относительно продаж ТГ 14.03-19.06
0,ТОВ1192853,33-72-00Ц0-29527,33-72-00Ц0-29527_СИ,1414381,СИ,СИ ЦЕПЬ,6157_СИ,925,35.0,ЦЕПЬ,...,3. Большой размах (>30%),3. Большой размах (>50%),2. Средний размах (20-50%),1.075351,3. Большой размах (>30%),Да или нет,0.300000,2. Средний размах (20-50%),0.069670,0.220238
1,ТОВ1192856,33-72-00Ч0-29527,33-72-00Ч0-29527_СИ,1077949,СИ,СИ ЦЕПЬ,6157_СИ,925,35.0,ЦЕПЬ,...,3. Большой размах (>30%),3. Большой размах (>50%),2. Средний размах (20-50%),1.043093,3. Большой размах (>30%),Да или нет,0.300000,2. Средний размах (20-50%),0.093737,0.238095
2,ТОВ1192860,33-72-00Я0-29527,33-72-00Я0-29527_СИ,1352153,СИ,СИ ЦЕПЬ,6157_СИ,925,40.0,ЦЕПЬ,...,3. Большой размах (>30%),3. Большой размах (>50%),2. Средний размах (20-50%),1.305221,3. Большой размах (>30%),Да или нет,0.400000,2. Средний размах (20-50%),0.045126,0.004464
3,ТОВ1193256,20-72-0000-28618,20-72-0000-28618_СИ,1183070,СИ,СИ КОЛЬЦО БК,6302_СИ,925,18.5,КОЛЬЦО,...,3. Большой размах (>30%),3. Большой размах (>50%),1. Небольшой размах (<10%),0.617122,3. Большой размах (>30%),Да или нет,0.000000,1. Небольшой размах (<20%),0.054518,0.012649
4,ТОВ1193276,20-72-0001-26768,20-72-0001-26768_СИ,2240197,СИ,СИ КОЛЬЦО ИФ,6310_СИ,925,20.0,КОЛЬЦО,...,3. Большой размах (>30%),3. Большой размах (>50%),2. Средний размах (20-50%),1.088819,3. Большой размах (>30%),Да или нет,0.211617,2. Средний размах (20-50%),0.071688,0.007440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46161,ТОВ1038824,1215014533-505,1215014533-505_СИ,3824868,СИ,СИ СЕРЬГИ ЦВ/К,6239_СИ,925,0.0,СЕРЬГИ,...,1. Небольшой размах (<10%),1. Небольшой размах (<10%),1. Небольшой размах (<10%),0.000000,1. Небольшой размах (<10%),Да или нет,0.000000,1. Небольшой размах (<20%),0.000037,NaN
46162,ТОВ0987476,1010013831-501,1010013831-501_СИ,2813001,СИ,СИ КОЛЬЦО ИФ,6310_СИ,925,18.0,КОЛЬЦО,...,1. Небольшой размах (<10%),1. Небольшой размах (<10%),1. Небольшой размах (<10%),0.000000,1. Небольшой размах (<10%),Да или нет,0.000000,1. Небольшой размах (<20%),0.000037,NaN
46163,ТОВ1090811,И3-416-23-01#БЛОК,И3-416-23-01#БЛОК_СИ,2213992,СИ,СИ АКСЕССУАР,6509_СИ,925,0.0,БУЛАВКА,...,1. Небольшой размах (<10%),1. Небольшой размах (<10%),1. Небольшой размах (<10%),0.000000,1. Небольшой размах (<10%),Да или нет,0.000000,1. Небольшой размах (<20%),0.000037,NaN
46164,ТОВ1006903,442800-0-0,442800-0-0_СИ,3444080,СИ,СИ БРАСЛЕТ БК Декор,6028_СИ,925,17.5,БРАСЛЕТ,...,1. Небольшой размах (<10%),1. Небольшой размах (<10%),1. Небольшой размах (<10%),0.000000,1. Небольшой размах (<10%),Да или нет,0.000000,1. Небольшой размах (<20%),0.000037,NaN


In [138]:

result.to_excel(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\result.xlsx', index=False)

In [ ]:
result.to_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\\в разрезе ID исправленный.xlsx', index=False)

In [ ]:
result['Товарное направление'].value_counts()

Товарное направление
ПДК    15220
ДК     13820
ИФ      8328
БК      4796
ЦБ      2443
Name: count, dtype: int64

In [ ]:
for i in result['Товарное направление'].unique():
    result[result['Товарное направление'] == i].to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\\Исследование цен для {i} в разрезе ID.xlsx', index=False) 

In [143]:
df = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\Исследование цен в разрезе ID для СИ.xlsx')

In [144]:
df

,Код товара,Артикул поставщика,Внутренний артикул,ID Сайта,Товарное направление,Товарная группа,Товарный кластер,Проба,Тип изделия 1,Тип изделия 2,...,Размах по весу,Размах по себестоимости,Размах по цене со скидками,"Цена за грамм размах, %",Размах по цене за грамм,Скидка разрешена,"Цена изделия на бирке размах, %",Размах цена изделия на бирке,Доля % остатка ID отосительно остатка ТГ,Доля % относительно продаж ТГ 14.03-19.06
0,ТОВ1192853,33-72-00Ц0-29527,33-72-00Ц0-29527_СИ,1414381,СИ,СИ ЦЕПЬ,6157_СИ,925.0,ЦЕПЬ,ТЕКСТИЛЬ,...,3. Большой размах (>30%),3. Большой размах (>50%),2. Средний размах (20-50%),1.075351,3. Большой размах (>30%),Да или нет,0.300000,2. Средний размах (20-50%),0.069670,0.220238
1,ТОВ1192856,33-72-00Ч0-29527,33-72-00Ч0-29527_СИ,1077949,СИ,СИ ЦЕПЬ,6157_СИ,925.0,ЦЕПЬ,ТЕКСТИЛЬ,...,3. Большой размах (>30%),3. Большой размах (>50%),2. Средний размах (20-50%),1.043093,3. Большой размах (>30%),Да или нет,0.300000,2. Средний размах (20-50%),0.093737,0.238095
2,ТОВ1192860,33-72-00Я0-29527,33-72-00Я0-29527_СИ,1352153,СИ,СИ ЦЕПЬ,6157_СИ,925.0,ЦЕПЬ,ТЕКСТИЛЬ,...,3. Большой размах (>30%),3. Большой размах (>50%),2. Средний размах (20-50%),1.305221,3. Большой размах (>30%),Да или нет,0.400000,2. Средний размах (20-50%),0.045126,0.004464
3,ТОВ1193256,20-72-0000-28618,20-72-0000-28618_СИ,1183070,СИ,СИ КОЛЬЦО БК,6302_СИ,925.0,КОЛЬЦО,БЕЗ КАМНЕЙ,...,3. Большой размах (>30%),3. Большой размах (>50%),1. Небольшой размах (<10%),0.617122,3. Большой размах (>30%),Да или нет,0.000000,1. Небольшой размах (<20%),0.054518,0.012649
4,ТОВ1193276,20-72-0001-26768,20-72-0001-26768_СИ,2240197,СИ,СИ КОЛЬЦО ИФ,6310_СИ,925.0,КОЛЬЦО,С ФИАНИТАМИ,...,3. Большой размах (>30%),3. Большой размах (>50%),2. Средний размах (20-50%),1.088819,3. Большой размах (>30%),Да или нет,0.211617,2. Средний размах (20-50%),0.071688,0.007440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46161,ТОВ1038824,1215014533-505,1215014533-505_СИ,3824868,СИ,СИ СЕРЬГИ ЦВ/К,6239_СИ,925.0,СЕРЬГИ,СО СТЕКЛОМ,...,1. Небольшой размах (<10%),1. Небольшой размах (<10%),1. Небольшой размах (<10%),0.000000,1. Небольшой размах (<10%),Да или нет,0.000000,1. Небольшой размах (<20%),0.000037,NaN
46162,ТОВ0987476,1010013831-501,1010013831-501_СИ,2813001,СИ,СИ КОЛЬЦО ИФ,6310_СИ,925.0,КОЛЬЦО,С ФИАНИТАМИ,...,1. Небольшой размах (<10%),1. Небольшой размах (<10%),1. Небольшой размах (<10%),0.000000,1. Небольшой размах (<10%),Да или нет,0.000000,1. Небольшой размах (<20%),0.000037,NaN
46163,ТОВ1090811,И3-416-23-01#БЛОК,И3-416-23-01#БЛОК_СИ,2213992,СИ,СИ АКСЕССУАР,6509_СИ,925.0,БУЛАВКА,С ФИАНИТАМИ,...,1. Небольшой размах (<10%),1. Небольшой размах (<10%),1. Небольшой размах (<10%),0.000000,1. Небольшой размах (<10%),Да или нет,0.000000,1. Небольшой размах (<20%),0.000037,NaN
46164,ТОВ1006903,442800-0-0,442800-0-0_СИ,3444080,СИ,СИ БРАСЛЕТ БК Декор,6028_СИ,925.0,БРАСЛЕТ,БЕЗ КАМНЕЙ,...,1. Небольшой размах (<10%),1. Небольшой размах (<10%),1. Небольшой размах (<10%),0.000000,1. Небольшой размах (<10%),Да или нет,0.000000,1. Небольшой размах (<20%),0.000037,NaN
